In [11]:
def convert_to_grayscale(img):
    # Check if the image is already in grayscale
    if len(img.shape) == 2:
        return img
    
    # Convert to grayscale
    grayscale_img = np.zeros_like(img[..., 0])  # Create an empty grayscale image
    
    # Iterate over each pixel and calculate the grayscale value
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            b, g, r = img[i, j]  # Get the BGR values of the pixel
            gray_value = int(0.299 * r + 0.587 * g + 0.114 * b)  # Calculate the grayscale value
            grayscale_img[i, j] = gray_value  # Assign the grayscale value to the corresponding pixel
    
    return grayscale_img

In [12]:
def manual_otsu_threshold(gray_image):
    # Calculate histogram
    hist = cv2.calcHist([gray_image], [0], None, [256], [0,256])
    hist_norm = hist.ravel() / hist.sum()
    # Calculate probabilities
    q = np.cumsum(hist_norm)
    bins = np.arange(256)

    # Initialize variables
    fn_min = np.inf
    thresh = -1
    # Iterate through all thresholds to find optimal one
    for i in range(1, 256):
        p1, p2 = np.hsplit(hist_norm, [i]) # probabilities
        q1, q2 = q[i], q[255] - q[i] # cum sum of classes
        b1, b2 = np.hsplit(bins, [i]) # weights

        # Skip calculation if q1 or q2 is zero
        if q1 == 0 or q2 == 0:
            continue

        # Finding means and variances
        m1, m2 = np.sum(p1 * b1) / q1, np.sum(p2 * b2) / q2
        v1, v2 = np.sum(((b1 - m1) ** 2) * p1) / q1, np.sum(((b2 - m2) ** 2) * p2) / q2

        # Calculate the minimum within-class variance
        fn = v1 * q1 + v2 * q2
        if fn < fn_min:
            fn_min = fn
            thresh = i

    # Binarize image
    ret, bin_img = cv2.threshold(gray_image, thresh, 255, cv2.THRESH_BINARY_INV)

    return ret, bin_img


In [13]:
def manual_dilate(binary_image, kernel, iterations=1):
    # Define the dimensions of the kernel
    kernel_height, kernel_width = kernel.shape
    
    # Get the dimensions of the input image
    image_height, image_width = binary_image.shape
    
    # Initialize an empty image to store the dilated result
    dilated_image = np.zeros_like(binary_image)
    
    # Pad the input image on all sides to handle edge cases
    padded_image = np.pad(binary_image, ((kernel_height//2, kernel_height//2), (kernel_width//2, kernel_width//2)), mode='constant', constant_values=0)
    
    # Iterate through each pixel in the padded image
    for i in range(image_height):
        for j in range(image_width):
            # Extract the region of interest (ROI) from the padded image using the kernel
            roi = padded_image[i:i+kernel_height, j:j+kernel_width]
            
            # Perform element-wise AND operation between the kernel and the ROI
            # If any pixel in the ROI is non-zero, set the corresponding pixel in the dilated image to 255
            if np.sum(roi & kernel) > 0:
                dilated_image[i, j] = 255
    
    # If iterations > 1, perform dilation repeatedly
    if iterations > 1:
        for _ in range(iterations - 1):
            dilated_image = manual_dilate(dilated_image, kernel, iterations=1)
    
    return dilated_image


def euclidean_distance_transform(bin_img):
    # Pad the binary image with zeros
    padded_img = np.pad(bin_img, 1, mode='constant', constant_values=0)

    # Initialize the output distance transform image
    distance_img = np.zeros_like(padded_img, dtype=np.float32)

    # Define the kernel for 3x3 neighborhood
    kernel = np.array([[1, 1, 1],
                       [1, 1, 1],
                       [1, 1, 1]])

    # Iterate over each pixel in the padded image
    for i in range(1, padded_img.shape[0] - 1):
        for j in range(1, padded_img.shape[1] - 1):
            if padded_img[i, j] == 0:  # Background pixel
                # Compute Euclidean distance
                dist = np.sqrt(np.sum((kernel * distance_img[i-1:i+2, j-1:j+2])**2))
                distance_img[i, j] = dist

    # Remove the padding to get the original size
    distance_img = distance_img[1:-1, 1:-1]

    return distance_img

def threshold_manual(dist_transform):
    # Calculate the threshold value
    threshold_value = 0.001 * np.max(dist_transform)
    
    # Apply the threshold manually
    thresholded_image = np.where(dist_transform > threshold_value, 255, 0).astype(np.uint8)

    return threshold_value, thresholded_image


In [14]:
import numpy as np

def connected_components_labeling_manual(image):
    # Initialize labels matrix
    labels = np.zeros_like(image)
    label_count = 1  # Start label count from 1, as 0 is reserved for background

    # Define dictionary to track equivalence between labels
    equivalence = {}

    # First pass
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] == 255:
                neighbors = []
                if i > 0:
                    neighbors.append(labels[i - 1, j])
                if j > 0:
                    neighbors.append(labels[i, j - 1])

                if len(neighbors) == 0:
                    labels[i, j] = label_count
                    label_count += 1
                else:
                    min_neighbor = min(neighbors)
                    labels[i, j] = min_neighbor
                    for neighbor in neighbors:
                        if neighbor != min_neighbor:
                            equivalence[neighbor] = min_neighbor

    # Second pass - Apply equivalence
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if labels[i, j] != 0:
                parent_label = labels[i, j]
                while parent_label in equivalence:
                    parent_label = equivalence[parent_label]
                labels[i, j] = parent_label

    # Count the number of connected components
    unique_labels = np.unique(labels)
    num_components = len(unique_labels) - 1  # Subtract 1 for the background label

    return num_components, labels

# Example usage:
# num_components_manual, labeled_image_manual = connected_components_labeling_manual(sure_fg)

#You're correct; the simple manual implementation provided may not yield the same results as OpenCV's cv2.connectedComponents() function due to its simplified approach. OpenCV's implementation is more complex and optimized for accuracy and efficiency.

In [15]:
import numpy as np

def watershed_manual(image, markers):
    # Initialize labels matrix
    labels = np.copy(markers)

    # Define queue for pixels to process
    queue = []

    # Define neighborhood kernel
    kernel = np.array([[1, 1, 1],
                       [1, 1, 1],
                       [1, 1, 1]])

    # Add marker pixels to the queue
    for i in range(markers.shape[0]):
        for j in range(markers.shape[1]):
            if markers[i, j] == -1:
                queue.append((i, j))
                labels[i, j] = 0

    # Process pixels in the queue
    while queue:
        # Get pixel coordinates from the queue
        i, j = queue.pop(0)

        # Explore neighbors
        for di in range(-1, 2):
            for dj in range(-1, 2):
                ni, nj = i + di, j + dj
                if 0 <= ni < image.shape[0] and 0 <= nj < image.shape[1]:
                    if labels[ni, nj] == -1:
                        labels[ni, nj] = labels[i, j]
                        queue.append((ni, nj))

    # Compute image gradients
    gradient_x = np.gradient(image)[0]
    gradient_y = np.gradient(image)[1]

    # Compute gradient magnitude
    gradient_magnitude = np.sqrt(gradient_x ** 2 + gradient_y ** 2)

    # Find local minima in gradient magnitude
    local_minima = np.zeros_like(gradient_magnitude, dtype=bool)
    for i in range(1, gradient_magnitude.shape[0] - 1):
        for j in range(1, gradient_magnitude.shape[1] - 1):
            neighbors = gradient_magnitude[i - 1:i + 2, j - 1:j + 2]
            local_minima[i, j] = np.all(gradient_magnitude[i, j] <= neighbors)

    # Get coordinates where local_minima is True
    minima_coords = np.where(local_minima)

    # Assign -1 to labels where local_minima is True
    labels[minima_coords[0], minima_coords[1]] = -1
     
    return labels

# Example usage:
# watershed_result = watershed_manual(img, markers)

